# Introduction

In this notebook, we will experiment with how user queries are handled in our veterinary information retrieval system. Several collections have already been set up in the Chroma database, allowing us to directly perform information retrieval without additional setup. This environment enables us to test and refine the process of transforming user input into actionable queries and retrieving relevant information from our knowledge base.

In [1]:
from langchain_experimental.open_clip import OpenCLIPEmbeddings
from langchain_chroma import Chroma

persist_directory = '../chroma/textbook_test_Nutrition'
id_key = "doc_id"

open_clip_embeddings = OpenCLIPEmbeddings(model_name="ViT-g-14", checkpoint="laion2b_s34b_b88k")

# Vectorstore for summaries (for similarity search)
vectorstore = Chroma(
    collection_name="summaries",
    persist_directory=persist_directory,
    embedding_function=open_clip_embeddings
)
# Persistent docstore for originals (all modalities)
docstore = Chroma(
    collection_name="originals",
    persist_directory=persist_directory,
    embedding_function=open_clip_embeddings
)

# Instantiate the retriever

class UnifiedRetriever:
        
        def __init__(self, vectorstore, docstore, id_key="doc_id"):
            self.vectorstore = vectorstore
            self.docstore = docstore
            self.id_key = id_key
            self._collection = docstore._collection

        def retrieve(self, query, k=5):
            results = self.vectorstore.similarity_search_with_score(query, k=k)
            output = []
            for doc, score in results:
                doc_id = doc.metadata.get(self.id_key)
                try:
                    original = self._collection.get(ids=[doc_id], include=["documents", "metadatas"])
                    original_doc = original["documents"][0] if original["documents"] else None
                    original_meta = original["metadatas"][0] if original["metadatas"] else None
                except Exception as e:
                    original_doc = None
                    original_meta = None
                output.append({
                    "summary": doc.page_content,
                    "original": original_doc,
                    "original_metadata": original_meta,
                    "summary_metadata": doc.metadata,
                    "score": score
                })
            return output

retriever = UnifiedRetriever(vectorstore, docstore, id_key=id_key)

# Handling User Input: Analysis Image

Let's use a image of a under weighted cat. This cat is considerablly skinny with bones showing. 

  ![Skinney Cat](./skinny_cat.jpg)

In [9]:
query = "What's going on with my cat? What should I do?" 
image_path = "./skinny_cat.jpg"

import base64
import ollama
import os

# --- Configuration for the image ---
# IMPORTANT: Adjust this path if your cat.jpg is in a different location
# image_model = "minicpm-v:8b" # Or "llava:7b" or another suitable vision model you have installed via Ollama
# image_model = "llava:7b" # 

#This instruct version, q8_0 weight format fits MacBook M1 Pro better
image_model = "llama3.2-vision:11b-instruct-q4_K_M" # 



# --- 1. Generate a textual summary of the image using an LLM ---
print(f" ⏳ Processing image: {image_path}")

image_summary = "Could not generate image summary." # Default in case of error
if not os.path.exists(image_path):
    print(f"Error: Image file not found at {image_path}. Please check the path.")
else:
    try:
        # Read and encode image in base64
        with open(image_path, 'rb') as f:
            image_data = base64.b64encode(f.read()).decode('utf-8')

        # Updated prompt for detailed image summarization
        image_summarization_prompt = """From a feline veterinary stand point, provide a highly detailed and objective 
                description of the image. Focus on all observable elements, actions, 
                objects, subjects, their attributes (e.g., color, size, texture), 
                their spatial relationships, and any discernible context or implied scene. 
                Also focus on all possible health issue.
                Describe any text present in the image. This description must be exhaustive 
                and purely factual, capturing every significant visual detail to serve as a 
                comprehensive textual representation for further analysis by another AI model. 
                If the image is entirely irrelevant or contains no discernible subject, 
                state "No relevant visual information."."""

        # Send image to ollama for vision model processing
        response = ollama.chat(
            model=image_model,
            messages=[
                {
                    'role': 'user',
                    'content': image_summarization_prompt,
                    'images': [image_data]
                }
            ]
        )
        image_summary = response['message']['content']
        print("--- Generated Image Summary ---")
        print(image_summary)

    except Exception as e:
        print(f"Error processing image with Ollama: {e}")

# This 'image_summary' can now be used along with your user's text query
# for retrieval or further processing in your RAG pipeline.

 ⏳ Processing image: ./skinny_cat.jpg
--- Generated Image Summary ---
The image depicts a cat standing next to a bowl of water, with its head resting on the edge of the bowl. The cat is positioned on a tiled floor, facing the camera, and appears to be in a state of poor health.

* **Cat**
	+ The cat is light-colored, with a pale yellowish-white coat.
	+ It has a thin and emaciated appearance, with visible ribs and a prominent spine.
	+ Its eyes are yellowish-green, and its ears are pointed and erect.
	+ The cat's fur is matted and unkempt, with visible signs of neglect.
* **Bowl**
	+ The bowl is light green and appears to be made of plastic.
	+ It is placed on the floor next to the cat, with the cat's head resting on the edge.
	+ The bowl is empty, with no water visible inside.
* **Floor**
	+ The floor is made of small, square tiles that are a light brown color.
	+ The tiles are arranged in a grid pattern, with a slight grout line between each tile.
	+ The floor appears to be dirty, wi

# Handling User Input: Refine Query

In [13]:
from langchain_ollama import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

# Define the LLM for query refinement (using the same model as your RAG chain if appropriate)
# Compressed, Distilled Qwen, Response often in CoT <think></think>
# query_refinement_model = ChatOllama(model="deepseek-r1:7b-qwen-distill-q8_0")

#compressed that fits m1 pro, use less RAM. No CoT
query_refinement_model = ChatOllama(model="llama3.2:3b")

# Prompt for query refinement
query_refinement_prompt = ChatPromptTemplate.from_template(
    """You are an intelligent assistant. Your task is to rephrase and expand the given user query \
into a more detailed and context-rich query that can be used to retrieve relevant information \
from a veterinary knowledge base. Use the provided image description to add visual context \
and relevant keywords to the refined query. Focus on adding relevant keywords, clarifying intent, \
and anticipating related information that might be helpful. The output should be a single, refined query.

Original query: {original_query}
Image description: {image_summary}"""
)

# Create the query refinement chain
query_refinement_chain = (
    {
        "original_query": RunnablePassthrough(),
        "image_summary": RunnablePassthrough()
    }
    | query_refinement_prompt
    | query_refinement_model
    | StrOutputParser()
)# --- Demonstration of query refinement and then retrieval with scores ---

# print(f"Original user query: {query}")
# print(f"Image Summary: {image_summary}")

refined_query = query_refinement_chain.invoke(
    {"original_query": query, "image_summary": image_summary}
)

print("-"*80)
print(f"Refined query: {refined_query}")


--------------------------------------------------------------------------------
Refined query: Refined Query:

"I'm concerned about my light-colored, pale yellowish-white domestic shorthair cat's health, as indicated by its thin and emaciated appearance, visible ribs, prominent spine, yellowish-green eyes, pointed and erect ears, matted and unkempt fur, and signs of neglect. The cat is positioned on a tiled floor with small, square tiles in a grid pattern, made of light brown material, which appears dirty with visible dust and dirt particles. Notably, the cat's food bowl, made of light green plastic, is empty and placed next to the cat, with no water visible inside. Given these visual cues, I would like to know:

1. What are the possible causes of my cat's poor health, malnutrition, and signs of neglect?
2. What steps can I take immediately to ensure my cat's immediate needs are met, such as providing food, water, and shelter?
3. Are there any specific veterinary conditions or disease

# Query Decomposition

In [16]:
from langchain_core.output_parsers import JsonOutputParser
# Prompt for query decomposition
query_decomposition_prompt = ChatPromptTemplate.from_template(
    """You are an intelligent assistant. Your task is to break down the given complex query
into a list of simpler, focused sub-queries. Each sub-query should be a standalone question
that can be used to retrieve specific information from a veterinary knowledge base.

Output ONLY a valid JSON array of strings, and nothing else. Do not include any explanations, markdown, or extra text.

Complex query: {refined_query}
"""
)

# Create the query decomposition chain
query_decomposition_chain = (
    query_decomposition_prompt  
    | query_refinement_model    
    | JsonOutputParser() 
)

# --- Demonstration of query decomposition ---

print(f"Original refined query: {refined_query[:300]} ....")

decomposed_queries = query_decomposition_chain.invoke({"refined_query": refined_query})

print("-" * 80)
# print(f"Decomposed queries:\n{decomposed_queries}")

print(f"There are {len(decomposed_queries)} queries after decomposition \n")
print(f"Here's a example of the first one: {decomposed_queries[0]}")


Original refined query: Refined Query:

"I'm concerned about my light-colored, pale yellowish-white domestic shorthair cat's health, as indicated by its thin and emaciated appearance, visible ribs, prominent spine, yellowish-green eyes, pointed and erect ears, matted and unkempt fur, and signs of neglect. The cat is positi ....
--------------------------------------------------------------------------------
There are 6 queries after decomposition 

Here's a example of the first one: What are the possible causes of thin and emaciated domestic shorthair cats?


# Contextual Retrievals 

In [17]:
# Assume decomposed_queries is a list of query strings
# and retriever is already instantiated

seen_doc_ids = set()
all_results = []

for query in decomposed_queries:
    results = retriever.retrieve(query, k=5)
    unique_results = []
    for res in results:
        doc_id = res.get('doc_id') or res.get('summary_metadata', {}).get('doc_id')
        if doc_id and doc_id not in seen_doc_ids:
            seen_doc_ids.add(doc_id)
            unique_results.append(res)
    if unique_results:
        all_results.append({
            "query": query,
            "results": unique_results
        })

# Summarize all_results
total_unique_docs = sum(len(entry['results']) for entry in all_results)
total_queries_with_results = len(all_results)
all_doc_ids = set()
for entry in all_results:
    for res in entry['results']:
        doc_id = res.get('doc_id') or res.get('summary_metadata', {}).get('doc_id')
        if doc_id:
            all_doc_ids.add(doc_id)

print(f"Total unique documents retrieved: {len(all_doc_ids)}")
print(f"Total queries with at least one unique result: {total_queries_with_results}")
print("Number of unique documents retrieved per query:")
for entry in all_results:
    print(f"  Query: {entry['query'][:60]}... -> {len(entry['results'])} unique docs")



INFO:backoff:Backing off send_request(...) for 0.6s (requests.exceptions.ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')))


Total unique documents retrieved: 10
Total queries with at least one unique result: 4
Number of unique documents retrieved per query:
  Query: What are the possible causes of thin and emaciated domestic ... -> 5 unique docs
  Query: How to provide food, water, and shelter to malnourished dome... -> 1 unique docs
  Query: Common veterinary conditions or diseases affecting domestic ... -> 3 unique docs
  Query: Signs of dehydration in domestic shorthair cats... -> 1 unique docs


In [ ]:
all_results[0]

# Directly Answering Query with Retrieved Info

In [20]:
# Combine all summaries from all_results into one context
all_context = "\n".join(
    res['summary']
    for entry in all_results
    for res in entry['results']
)


final_answer_prompt = ChatPromptTemplate.from_template(
    """You are a helpful veterinary assistant. Use the provided context to answer the 
    user's question as thoroughly and concisely as possible. Follow the following steps
    when giving an prompt answer to the user.
    1. Describe what was the main issue that you observed from the image summary?
    2. What could be the cause of the main issue?
    3. How can this main issue be solved?

    User's question: {query}

    Image Summary: {image_summary}

    Context:
    {context}

    Answer:"""
    )

final_answer_chain = (
    final_answer_prompt
    | ChatOllama(model="llama3.2:3b")  
    | StrOutputParser()
)

final_answer = final_answer_chain.invoke({
    "query": query,
    "image_summary": image_summary,
    "context": all_context
})

print("Final Answer:")
print(final_answer)

Final Answer:
Here's a thorough and concise response as a veterinary assistant:

1. **Main Issue:** The main issue observed in the image summary is the cat's poor health, with visible signs of neglect and malnutrition. The cat appears thin and emaciated, with visible ribs and a prominent spine, indicating potential malnutrition or starvation.

2. **Possible Cause:** Based on the provided context, it seems likely that the cat's diet has been inadequate or lacking in essential nutrients. The fact that the bowl is empty and the floor is dirty suggests that the cat may not have access to clean water or food for an extended period. Additionally, the presence of mats and unkempt fur indicates a lack of proper grooming and hygiene.

3. **Solution:** To solve this main issue, it would be essential to provide the cat with a nutritious diet that meets its protein and nutrient requirements. This may involve switching to a high-quality premium brand cat food that has a higher meat content and fewe

# Tool Calling Chain of Thought Way To Answer Query

Previously, We used llama3.2:3b model to answer the query with image summary and retrieved info giving to it. However, the answer can be incomprehensive and missing the target. Here we will try to use reasoning model that outputs chain of thought(CoT) that can solve this issue. For example:
1. deepseek-r1 7b 8b 
2. qwen3 4b 8b


In [ ]:
# Have model to think of a plan to answer query that helps user to understand the cause, 
# possible underlying issue, recommended next steps, etc.

# If context not enough, fetch info on Wikipedia, or use Tavily

# Let it pause and think, what info gap is there? Does it need more info from the owner? 
# i.e. "is vaccine up-to-date?", "is eating and drinking ok?", "is pooping and peeing ok?",...